In [1]:
import sys
sys.version

Out[ 1 ]: '3.5.2 (default, Nov 23 2017, 16:37:01) \n[GCC 5.4.0 20160609]'

In [2]:
import sys, os
import requests, re
from bs4 import BeautifulSoup
import json
import time, datetime
from urllib.parse import urlparse

Generate a Databricks Token for Azure Cloud Shell Access <br>
https://docs.azuredatabricks.net/api/latest/authentication.html#token-management <br>
From the Azure Cloud Bash Shell, install Databricks CLI <br>
https://docs.microsoft.com/en-us/azure/azure-databricks/databricks-cli-from-azure-cloud-shell<br>

Create a Databricks Secrets Scope for "azure_ml" <br>
<code>databricks secrets create-scope --scope azure_ml</code>

Gather Tokens for the Azure Cognitive Tokens and store them in the "azure_ml" scope <br>
<code>databricks secrets put --scope azure_ml --key custom_vision_training_key --string-value TRAINING_KEY_FROM_CUSTOM_VISION </code>
<code>databricks secrets put --scope azure_ml --key custom_vision_prediction_key --string-value PREDICTION_KEY_FROM_CUSTOM_VISION </code>

In [4]:
custom_vision_training_key = dbutils.preview.secret.get(scope = "azure_ml", key = "custom_vision_training_key")
custom_vision_prediction_key = dbutils.preview.secret.get(scope = "azure_ml", key = "custom_vision_prediction_key")

In [5]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry

trainer = training_api.TrainingApi(custom_vision_training_key)

In [6]:
projects = trainer.get_projects()
project = next((project for project in projects if project.name == "ShoeStyleTagger"), None)
if project == None:
  # Create a new project
  project = trainer.create_project("ShoeStyleTagger")

Out[ 13 ]: '7623650c-ce4d-49dd-bbf0-1eacd870ca4c'

In [7]:
demographic_tag = trainer.create_tag(project.id, "kids")
gender_tag = trainer.create_tag(project.id, "girls")
category_tag = trainer.create_tag(project.id, "shoes")
style_tag_boots = trainer.create_tag(project.id, "boots")
style_tag_dress = trainer.create_tag(project.id, "dress")
style_tag_sneakers = trainer.create_tag(project.id, "sneakers")

In [8]:
requestPaths = {
    '/shop/kids-clothes/kids-shoes/Gender,Shoe_type,Productsperpage/Girls,Dress%20Shoes,120?id=48561':[demographic_tag.id, gender_tag.id, category_tag.id, style_tag_dress.id],
    '/shop/kids-clothes/kids-shoes/Gender,Shoe_type,Productsperpage/Girls,Boots,120?id=48561':[demographic_tag.id, gender_tag.id, category_tag.id, style_tag_boots.id],
    '/shop/kids-clothes/kids-shoes/Gender,Shoe_type,Productsperpage/Girls,Boots%7CSneakers,120?id=48561':[demographic_tag.id, gender_tag.id, category_tag.id, style_tag_sneakers.id]
    }

In [9]:
for requestPath, tagList in requestPaths.items() :
    headers = {
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'path': requestPath,
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'authority': 'www.macys.com',
        'referer': 'https://www.macys.com/'
    }
    requestURL = 'https://www.macys.com/' + requestPath
    html_resp = requests.get(requestURL, headers=headers)
    if html_resp.status_code == 200:
        c = html_resp.content
        soup = BeautifulSoup(c, from_encoding="utf-8")
        product_cards = soup.find_all('li', class_='productThumbnailItem')
        for product_card in product_cards:
            product_img_link = product_card.find('div', class_='productThumbnailImage').find('img', class_='thumbnailImage').attrs['src']
            trainer.create_images_from_urls(project.id, [ImageUrlCreateEntry(url=product_img_link,tag_ids=tagList)])
            print(product_img_link)

https://slimages.macysassets.com/is/image/MCY/products/0/optimized/8587920_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/2/optimized/3925082_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/7/optimized/2532817_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/7/optimized/8647287_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/5/optimized/8440925_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/0/optimized/2532820_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/1/optimized/9909021_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/8/optimized/9909818_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/5/optimized/9997445_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/6/optimized/9997956_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/1/optimized/9158411_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/0/optimized/9948100_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/1/optimized/3029771_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/7/optimized/8953487_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/4/optimized/9266324_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/1/optimized/9947911_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/3/optimized/2532783_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/4/optimized/8850384_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/5/optimized/8587915_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/7/optimized/9948067_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/3/optimized/8825533_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/8/optimized/9909708_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/7/optimized/9947817_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/6/optimized/9909776_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/4/optimized/9909724_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/0/optimized/9909800_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/8/optimized/9909658_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/7/optimized/9146827_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/0/optimized/9996010_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/4/optimized/9145084_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/8/optimized/8953198_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/4/optimized/9146834_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/1/optimized/9146841_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/0/optimized/9146820_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/1/optimized/8952741_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/3/optimized/8578733_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/3/optimized/8610693_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/7/optimized/8610667_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/2/optimized/8824032_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/8/optimized/8930988_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/0/optimized/9123010_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/4/optimized/9997964_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/0/optimized/8587920_fpx.tif?$thumb$
https://slimages.macysassets.com/is/image/MCY/products/3/optimized/3447583_fpx.tif?$thu

In [10]:
import time

print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

# The iteration is now trained. Make it the default project endpoint
trainer.update_iteration(project.id, iteration.id, is_default=True)
print ("Done!")

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!

Try this image for prediction: "https://tse2.mm.bing.net/th?id=OIP.QpxuXic1_vcHKsSO7cwkQwHaHa&pid=Api"
<img src="https://tse2.mm.bing.net/th?id=OIP.QpxuXic1_vcHKsSO7cwkQwHaHa&pid"></img>

In [12]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

# Now there is a trained endpoint that can be used to make a prediction

predictor = prediction_endpoint.PredictionEndpoint(custom_vision_prediction_key)

test_img_url = "https://tse2.mm.bing.net/th?id=OIP.QpxuXic1_vcHKsSO7cwkQwHaHa&pid=Api"
results = predictor.predict_image_url(project.id, iteration.id, url=test_img_url)

# Alternatively, if the images were on disk in a folder called Images alongside the sample.py, then
# they can be added by using the following.
#
# Open the sample image and get back the prediction results.
# with open("Images\\test\\test_image.jpg", mode="rb") as test_data:
#     results = predictor.predict_image(project.id, test_data, iteration.id)

# Display the results.
for prediction in results.predictions:
    print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))

shoes: 99.99%
	girls: 99.99%
	kids: 99.99%
	sneakers: 99.46%
	boots: 7.64%
	dress: 0.04%